In [25]:
import pandas as pd
from myUtils import list_dump
path_dir = "C:/Users/ping/Desktop/my_yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
### edit file name to Barchart 52-week_new-highs ####
path_fn_52wkHi = "C:/Users/ping/Downloads/all-us-exchanges-52-week-new-highs-08-24-2022.csv"
#######################

In [26]:
df = pd.read_csv(path_fn_52wkHi)  # read Barchart 52-week_new-highs
# drop last row (e.g. Downloaded from Barchart.com as of 08-24-2022 06:21pm CDT)
df.drop(df.tail(1).index,inplace=True)
df_shape_orig = df.shape
# create column of Last * Volume
df['Last*Vol'] = df.Last * df.Volume

In [27]:
# apply filter Last*Vol > $10mil, Last > $10
df = df[df['Last*Vol'] > 10000000] 
df = df[df['Last'] > 10] 
df_shape_after = df.shape
row_deleted = df_shape_orig[0] - df_shape_after[0]
print(f'number of symbols removed by filter: {row_deleted}')
print(f'number of symbols remain:            {df_shape_after[0]}')

number of symbols removed by filter: 24
number of symbols remain:            38


In [28]:
# write symbols to text file
symbols_52wkHi = df['Symbol'].to_list()
list_dump(symbols_52wkHi, path_data_dump, 'symbols_52wkHi.txt', verbose=True)


+ def list_dump(list_to_dump, path_list_dump, filename)

Wrote list to C:/Users/ping/Desktop/my_yfinance/VSCode_dump/symbols_52wkHi.txt
- def list_dump(list_to_dump, path_list_dump, filename)

------------------------------------------------------------------------------



In [29]:
# columns for df_52wkHi
myCols = ['symbol',
 'quoteType',
 'longName',
  'sector',
 'industry',
 'marketCap',
 'revenueGrowth',
 'ebitdaMargins',
 'earningsGrowth',
 'earningsQuarterlyGrowth',
  'previousClose',
  'volume',
 'fiftyTwoWeekHigh',
 '52WeekChange',
 'revenuePerShare',
 'forwardEps',
 'trailingEps',
 'sharesOutstanding']

In [30]:
import yfinance as yf
# get yf.Ticker(symbol).info for keys in myCols for 52wkHi symbols 
df_list = []
for symbol in symbols_52wkHi:
    df_list.append(pd.DataFrame([yf.Ticker(symbol).info]))
df = pd.concat(df_list)
df = df[myCols].copy()

In [31]:
from datetime import date
# put today's date in the date column
today = str(date.today())
df['date'] = today

In [32]:
from myUtils import pickle_dump
# sort df_52wkHi with highest revenueGrowth on the 1st row
df_52wkHi = df.sort_values(by=['revenueGrowth', 'ebitdaMargins', 'earningsQuarterlyGrowth', 'earningsGrowth' ], ascending=False)
pickle_dump(df_52wkHi, path_data_dump, 'df_52wkHi')

In [33]:
# apply filter revenueGrowth > .50 & ebitdaMargins > .25
df_filtered = df_52wkHi.query('revenueGrowth > .50 & ebitdaMargins > .25')
df_filtered


,symbol,quoteType,longName,sector,industry,marketCap,revenueGrowth,ebitdaMargins,earningsGrowth,earningsQuarterlyGrowth,previousClose,volume,fiftyTwoWeekHigh,52WeekChange,revenuePerShare,forwardEps,trailingEps,sharesOutstanding,date
0,VET,EQUITY,Vermilion Energy Inc.,Energy,Oil & Gas E&P,4651517952,1.228,0.93280,-0.455,-0.432,27.66,3090419,28.73,None,14.199,0.61,1.541,162300000,2022-08-24
0,EQNR,EQUITY,Equinor ASA,Energy,Oil & Gas Integrated,137418915840,1.094,0.53756,2.535,2.487,40.92,3246696,42.10,None,40.068,3.18,-0.913,3232120064,2022-08-24
0,OXY,EQUITY,Occidental Petroleum Corporation,Energy,Oil & Gas E&P,69964521472,0.577,0.53054,None,None,73.79,39632291,75.69,None,30.999,-1.05,-17.063,933980992,2022-08-24


In [34]:
symbols_52wkHi = df_filtered['symbol'].tolist()  # convert pd.Series to list
pickle_dump(symbols_52wkHi, path_data_dump, 'symbols_52wkHi')

In [36]:
# plot symbols
from myUtils import yf_candlestick
plot_days = '252d'
for symbol in symbols_52wkHi:
  df = yf.Ticker(symbol).history(period=plot_days)
  df.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)  # drop Div. and Stock Splits columns
  df.columns = ['open', 'high', 'low', 'close', 'volume']  # rename columns Open ... Volume to open ... volume
  cache_plot = yf_candlestick(symbol, df, plot_chart=True)

In [39]:
# print symbols data
from myUtils import yf_print_symbol_data
cache_print = yf_print_symbol_data(symbols_52wkHi)

VET
                 Open       High        Low  Close   Volume  Dividends  Stock Splits
Date                                                                                
2022-08-18  25.020000  25.370001  24.830000  25.33  2017200          0             0
2022-08-19  25.000000  25.870001  24.780001  25.51  1999900          0             0
2022-08-22  25.209999  26.719999  24.950001  26.68  2775500          0             0
2022-08-23  27.360001  28.299999  27.129999  27.66  3586200          0             0
2022-08-24  27.709999  28.730000  27.430000  28.66  3089600          0             0

symbol                    VET
quoteType                 EQUITY
longName                  Vermilion Energy Inc.
industry                  Oil & Gas E&P
sector                    Energy
marketCap                      4.700B
revenueGrowth                  1.120
earningsGrowth                -0.216
earningsQuarterlyGrowth       -0.196
ebitdaMargins                  0.552
previousClose                 